In [0]:
%sql
SHOW GRANTS TO CURRENT_USER;

In [0]:
display(dbutils.fs.ls("/mnt/raw-ingest"))

In [0]:
from pyspark.sql.functions import input_file_name, lit

path = "/mnt/raw-ingest/finance_invoice_data.csv"

df_finance = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(path)
        .withColumn("source_file", input_file_name())
        .withColumn("ingestion_type", lit("finance_invoices"))
)

df_finance.write.format("delta").mode("overwrite").save("/mnt/delta/bronze/finance_invoices")


In [0]:
display(df_finance)

In [0]:
raw_df = spark.read.option("header", True).csv("/mnt/raw-ingest/finance_invoice_data.csv")
display(raw_df)


In [0]:
bronze_path = "/mnt/external-ingest/bronze/finance_invoice_data"


raw_df.write.format("delta").mode("overwrite").save(bronze_path)



In [0]:
bronze_df = spark.read.format("delta").load(bronze_path)
display(bronze_df)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS bronze_finance_invoice_data
USING DELTA
LOCATION '{bronze_path}'
""")


In [0]:
%sql
SELECT * FROM bronze_finance_invoice_data LIMIT 100


In [0]:
spark.table("bronze_finance_invoice_data").printSchema()

In [0]:
%sql
DESCRIBE bronze_finance_invoice_data


In [0]:
%sql
DESCRIBE HISTORY bronze_finance_invoice_data


In [0]:
%sql
VACUUM bronze_finance_invoice_data RETAIN 168 HOURS;

In [0]:
%sql
OPTIMIZE bronze_finance_invoice_data;


In [0]:
/mnt/delta/bronze/finance_invoices